In [585]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import boxcox
from collections import Counter
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
import warnings

In [423]:
# loading the imdb_full_info_df and inflation_date
imdb_full_info_df = pd.read_csv('imdb_full_info_df.csv')
inflation_df = pd.read_csv('inflation_data.csv')

In [424]:
inflation_df = inflation_df.sort_values('year', ascending=False)
inflation_df.drop(20, axis=0, inplace=True)
inflation_df.reset_index(inplace=True)
inflation_df.drop(columns=['index'], axis=1, inplace=True)

In [425]:
for i in range(0, len(inflation_df)):
    if i ==0:
        inflation_df['dollar'][i] = 1
    else:
        inflation_df['dollar'][i] = inflation_df['dollar'][i-1]*(1+inflation_df['inflation'][i-1])

C:\Users\Bosco\AppData\Local\Temp/ipykernel_32164/3125064047.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inflation_df['dollar'][i] = 1
C:\Users\Bosco\AppData\Local\Temp/ipykernel_32164/3125064047.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inflation_df['dollar'][i] = inflation_df['dollar'][i-1]*(1+inflation_df['inflation'][i-1])


In [431]:
# storing the year and compounded rate in a dictionary
inflation_dict = pd.Series(inflation_df.dollar.values, index=inflation_df.year).to_dict()

In [432]:
# adjusting inflation for budget and worldwide_gross
budget_adj = []
worldwide_gross_adj = []

for i in range(0,len(imdb_full_info_df)):
    release_year = int(imdb_full_info_df['release_date'][i].split('-')[0])
    inflation_comp = inflation_dict[release_year]
    budget_adj.append(round(imdb_full_info_df['budget'][i]*inflation_comp))
    worldwide_gross_adj.append(round(imdb_full_info_df['worldwide_gross'][i]*inflation_comp))

imdb_full_info_df['budget_adj'] = budget_adj
imdb_full_info_df['worldwide_gross_adj'] = worldwide_gross_adj

In [441]:
imdb_full_info_df

,release_date,budget,domestic_gross,worldwide_gross,title,imdb_rating,cast_1,cast_2,cast_3,movie_url,imdb_id,director,rating_check,runtime,genre,director.1,budget_adj,worldwide_gross_adj
0,2019-04-23,400000000,858373000,2797800564,Avengers: Endgame,8.4,Robert Downey Jr.,Chris Evans,Mark Ruffalo,https://www.imdb.com/title/tt4154796/,tt4154796,Anthony Russo,8.4,181,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",Anthony Russo,400000000,2797800564
1,2011-05-20,379000000,241071802,1045713802,Pirates of the Caribbean: On Stranger Tides,6.6,Johnny Depp,Penélope Cruz,Ian McShane,https://www.imdb.com/title/tt1298650/,tt1298650,Rob Marshall,6.6,137,"['Action', 'Adventure', 'Fantasy']",Rob Marshall,430707129,1188380976
2,2015-04-22,365000000,459005868,1395316979,Avengers: Age of Ultron,7.3,Robert Downey Jr.,Chris Evans,Mark Ruffalo,https://www.imdb.com/title/tt2395427/,tt2395427,Joss Whedon,7.3,141,"['Action', 'Adventure', 'Sci-Fi']",Joss Whedon,393679419,1504952267
3,2015-12-16,306000000,936662225,2064615817,Star Wars Episode VII: The Force Awakens,7.9,Daisy Ridley,John Boyega,Oscar Isaac,https://www.imdb.com/title/tt2488496/,tt2488496,J.J. Abrams,7.9,138,"['Action', 'Adventure', 'Sci-Fi']",J.J. Abrams,330043568,2226840424
4,2018-04-25,300000000,678815482,2048359754,Avengers: Infinity War,8.5,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,https://www.imdb.com/title/tt4154756/,tt4154756,Anthony Russo,8.5,149,"['Action', 'Adventure', 'Sci-Fi']",Anthony Russo,305280000,2084410886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3309,2011-09-30,5000000,223838,5476793,Tucker & Dale vs. Evil,7.5,Tyler Labine,Alan Tudyk,Katrina Bowden,https://www.imdb.com/title/tt1465522/,tt1465522,Eli Craig,7.5,89,"['Comedy', 'Horror']",Eli Craig,5682152,6223994
3310,2007-11-16,5000000,65388,861325,Redacted,6.1,Patrick Carroll,Rob Devaney,Izzy Diaz,https://www.imdb.com/title/tt0937237/,tt0937237,Brian De Palma,6.1,90,"['Crime', 'Thriller', 'War']",Brian De Palma,6164349,1061902
3311,2014-10-17,5000000,56001,584499,Rudderless,7.4,Billy Crudup,Anton Yelchin,Felicity Huffman,https://www.imdb.com/title/tt1798243/,tt1798243,William H. Macy,7.4,105,"['Comedy', 'Drama', 'Music']",William H. Macy,5399340,631182
3312,2012-05-17,5000000,0,9109597,Mientras duermes,7.2,Luis Tosar,Marta Etura,Alberto San Juan,https://www.imdb.com/title/tt1437358/,tt1437358,Jaume Balagueró,7.2,102,"['Crime', 'Drama', 'Thriller']",Jaume Balagueró,5566917,10142474


In [439]:
# extracting runtime values
runtime = []
for i in imdb_full_info_df['runtime']:
    runtime.append(int(i[2:][:-2]))
    
imdb_full_info_df['runtime'] = runtime

In [442]:
# dropping irrelevant columns
imdb_full_info_df.drop(columns=['release_date', 'domestic_gross', 'title', 'cast_1', 'cast_2', 'cast_3', 'movie_url', 'imdb_id', 'director', 'director.1','rating_check'], inplace=True)

In [443]:
imdb_full_info_df

,budget,worldwide_gross,imdb_rating,runtime,genre,budget_adj,worldwide_gross_adj
0,400000000,2797800564,8.4,181,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",400000000,2797800564
1,379000000,1045713802,6.6,137,"['Action', 'Adventure', 'Fantasy']",430707129,1188380976
2,365000000,1395316979,7.3,141,"['Action', 'Adventure', 'Sci-Fi']",393679419,1504952267
3,306000000,2064615817,7.9,138,"['Action', 'Adventure', 'Sci-Fi']",330043568,2226840424
4,300000000,2048359754,8.5,149,"['Action', 'Adventure', 'Sci-Fi']",305280000,2084410886
...,...,...,...,...,...,...,...
3309,5000000,5476793,7.5,89,"['Comedy', 'Horror']",5682152,6223994
3310,5000000,861325,6.1,90,"['Crime', 'Thriller', 'War']",6164349,1061902
3311,5000000,584499,7.4,105,"['Comedy', 'Drama', 'Music']",5399340,631182
3312,5000000,9109597,7.2,102,"['Crime', 'Drama', 'Thriller']",5566917,10142474


In [444]:
# one hot encoding for genre (multiple values)
x = imdb_full_info_df['genre'].apply(eval)
x = x.apply(Counter)
x = pd.json_normalize(x)
x = x.replace(np.nan, 0)
x = x.astype('int')

In [447]:
# merging two dataframes
model1_df = pd.concat([imdb_full_info_df, x], axis=1)
model1_df

,budget,worldwide_gross,imdb_rating,runtime,genre,budget_adj,worldwide_gross_adj,Action,Adventure,Drama,...,Crime,Western,History,Sport,Horror,Music,War,Biography,Documentary,Short
0,400000000,2797800564,8.4,181,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",400000000,2797800564,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,379000000,1045713802,6.6,137,"['Action', 'Adventure', 'Fantasy']",430707129,1188380976,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,365000000,1395316979,7.3,141,"['Action', 'Adventure', 'Sci-Fi']",393679419,1504952267,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,306000000,2064615817,7.9,138,"['Action', 'Adventure', 'Sci-Fi']",330043568,2226840424,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,300000000,2048359754,8.5,149,"['Action', 'Adventure', 'Sci-Fi']",305280000,2084410886,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3309,5000000,5476793,7.5,89,"['Comedy', 'Horror']",5682152,6223994,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3310,5000000,861325,6.1,90,"['Crime', 'Thriller', 'War']",6164349,1061902,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3311,5000000,584499,7.4,105,"['Comedy', 'Drama', 'Music']",5399340,631182,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3312,5000000,9109597,7.2,102,"['Crime', 'Drama', 'Thriller']",5566917,10142474,0,0,1,...,1,0,0,0,0,0,0,0,0,0


In [315]:
# NOT USED!!
# one hot encoding for genre (multiple values) - alternative
genre = []
for i in imdb_full_info_df['genre']:
    genre.append((i[2:][:-2]).replace("'","").replace(', ',','))
    
imdb_full_info_df['genre'] = genre

# finding unique values for genre
genre_cat = imdb_full_info_df['genre'].value_counts().reset_index()
genre_cat.head

genre_cat.rename(columns={'index':'genre_name', 'genre':'count'}, inplace=True)

genre_cat['genre_name'] = genre_cat['genre_name'].str.split(',')

df1 = []
for i in range(genre_cat.shape[0]):
    for j in genre_cat['genre_name'][i]:
        df1.append(j)
        
# creating a list of unique genres
list1 = list(set(df1))
list1

# creating a new dataframe with genre columns
df2 = pd.DataFrame(columns=list1)

# merging two dataframes
df3 = pd.concat([imdb_full_info_df, df2], axis=1)
df3

# filling binary values for genre columns
for i in range(0, len(df3)):
    for j in range(0, len(list1)):
        if df3.columns[5+j]in df3['genre'][i]:
            df3[df3.columns[5+j]][i] = 1
        else:
            df3[df3.columns[5+j]][i] = 0

### 1. Modelling - Linear Regression
#### budget-adjusted, imdb-rating, imdb-runtime, imdb-genre

In [448]:
model1_df.drop(['genre', 'budget', 'worldwide_gross'], axis=1, inplace=True)
model1_df

,imdb_rating,runtime,budget_adj,worldwide_gross_adj,Action,Adventure,Drama,Sci-Fi,Fantasy,Thriller,...,Crime,Western,History,Sport,Horror,Music,War,Biography,Documentary,Short
0,8.4,181,400000000,2797800564,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6.6,137,430707129,1188380976,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,7.3,141,393679419,1504952267,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7.9,138,330043568,2226840424,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8.5,149,305280000,2084410886,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3309,7.5,89,5682152,6223994,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3310,6.1,90,6164349,1061902,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
3311,7.4,105,5399340,631182,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3312,7.2,102,5566917,10142474,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0


In [588]:
def modelling(X, y, z):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 100)
    std_scaler = StandardScaler().fit(X_train)
    X_train_scaled = std_scaler.transform(X_train)
    X_test_scaled = std_scaler.transform(X_test)
    model = z
    model.fit(X_train_scaled, y_train)
    print('Train score is: '+str(model.score(X_train_scaled, y_train)))
    print('Test score is: '+str(model.score(X_test_scaled, y_test)))
    Adj_R2 = 1-((1-model.score(X_test_scaled, y_test))*(len(y_test)-1)/((len(y_test)-X_test.shape[1]-1)))
    print('Adjusted R2 is: '+str(Adj_R2))
    features_importance = pd.DataFrame(data = {
    'Attribute': X_train.columns,
    'Importance': abs(model.coef_)
    })
    features_importance = features_importance.sort_values(by='Importance', ascending=False)
    return features_importance

In [589]:
modelling(model1_df.drop('worldwide_gross_adj', axis=1), model1_df['worldwide_gross_adj'], LinearRegression())

Train score is: 0.6141719788796122
Test score is: 0.5959864071414929
Adjusted R2 is: 0.5801303006713787


,Attribute,Importance
2,budget_adj,1.609756e+08
0,imdb_rating,5.643034e+07
5,Drama,2.723804e+07
4,Adventure,1.505320e+07
1,runtime,1.338086e+07
19,Horror,1.245075e+07
16,Western,1.214529e+07
8,Thriller,1.129878e+07
3,Action,8.886172e+06
21,War,8.883672e+06


In [604]:
modelling(model1_df.drop('worldwide_gross_adj', axis=1), model1_df['worldwide_gross_adj'], Ridge(alpha=100.0))

Train score is: 0.6132132037502929
Test score is: 0.5975317939557374
Adjusted R2 is: 0.5817363384594947


,Attribute,Importance
2,budget_adj,1.515177e+08
0,imdb_rating,5.391711e+07
5,Drama,2.718846e+07
1,runtime,1.664271e+07
4,Adventure,1.658423e+07
16,Western,1.160360e+07
8,Thriller,1.109928e+07
19,Horror,1.085583e+07
21,War,8.903713e+06
17,History,8.793743e+06


### 2. Modelling - Linear Regression
#### budget-adjusted, imdb-rating, imdb-runtime, imdb-genre, rt-critic-rating, rt-audience-rating

In [552]:
# loading the Rotten Tomatoes data
final_v0_df = pd.read_csv('final_v0_df.csv')

In [577]:
rt_df = final_v0_df[['tomatometer_score', 'audience_score_rt']]

In [578]:
model2_df = pd.concat([model1_df, rt_df], axis=1)

In [579]:
model2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3314 entries, 0 to 3313
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   imdb_rating          3314 non-null   float64
 1   runtime              3314 non-null   int64  
 2   budget_adj           3314 non-null   int64  
 3   worldwide_gross_adj  3314 non-null   int64  
 4   Action               3314 non-null   int32  
 5   Adventure            3314 non-null   int32  
 6   Drama                3314 non-null   int32  
 7   Sci-Fi               3314 non-null   int32  
 8   Fantasy              3314 non-null   int32  
 9   Thriller             3314 non-null   int32  
 10  Animation            3314 non-null   int32  
 11  Family               3314 non-null   int32  
 12  Musical              3314 non-null   int32  
 13  Comedy               3314 non-null   int32  
 14  Romance              3314 non-null   int32  
 15  Mystery              3314 non-null   i

In [580]:
model2_df.isna().sum()

imdb_rating             0
runtime                 0
budget_adj              0
worldwide_gross_adj     0
Action                  0
Adventure               0
Drama                   0
Sci-Fi                  0
Fantasy                 0
Thriller                0
Animation               0
Family                  0
Musical                 0
Comedy                  0
Romance                 0
Mystery                 0
Crime                   0
Western                 0
History                 0
Sport                   0
Horror                  0
Music                   0
War                     0
Biography               0
Documentary             0
Short                   0
tomatometer_score      37
audience_score_rt      10
dtype: int64

In [581]:
model2_df.dropna(inplace=True)

In [583]:
linear_regression(model2_df.drop('worldwide_gross_adj', axis=1), model2_df['worldwide_gross_adj'], LinearRegression())

Train score is: 0.6235407654871908
Test score is: 0.5858708958261065
Adjusted R2 is: 0.5680375851200536


,Attribute,Importance
2,budget_adj,1.627051e+08
0,imdb_rating,3.261025e+07
5,Drama,3.188232e+07
26,audience_score_rt,2.015183e+07
4,Adventure,1.551334e+07
3,Action,1.457486e+07
1,runtime,1.409389e+07
25,tomatometer_score,1.304283e+07
21,War,1.139251e+07
19,Horror,1.044221e+07


### 3. Modelling - Linear Regression
#### budget-adjusted, imdb-rating, imdb-runtime, imdb-genre, rt-critic-rating, rt-audience-rating, rt-language, rt-mpa-rating

In [553]:
final_v0_df['MPA_Ratings'].value_counts().index

Index(['PG-13', 'R', 'PG', 'G', 'NC-17', 'TVMA', 'PG ', 'PG-13 ', 'R ', 'NR',
       'PG_13'],
      dtype='object')

In [554]:
final_v0_df['MPA_Ratings'] = final_v0_df['MPA_Ratings'].str.strip()

In [555]:
final_v0_df['MPA_Ratings'].value_counts()

PG-13    1310
R        1303
PG        497
G          77
NC-17       3
TVMA        2
NR          1
PG_13       1
Name: MPA_Ratings, dtype: int64

In [556]:
final_v0_df['MPA_Ratings'].replace({'PG_13':'PG-13'}, inplace=True)

In [557]:
final_v0_df['MPA_Ratings'].str.strip().value_counts()

PG-13    1311
R        1303
PG        497
G          77
NC-17       3
TVMA        2
NR          1
Name: MPA_Ratings, dtype: int64

In [537]:
final_v0_df['MPA_Ratings'].str.strip(' ')

0       PG-13
1       PG-13
2       PG-13
3       PG-13
4       PG-13
        ...  
3309        R
3310        R
3311        R
3312      NaN
3313      NaN
Name: MPA_Ratings, Length: 3314, dtype: object

In [560]:
rt2_df = final_v0_df[['tomatometer_score', 'audience_score_rt', 'original_language', 'MPA_Ratings']]

In [565]:
# merging two dataframes
model3_df = pd.concat([model1_df, rt2_df], axis=1)
model3_df

,imdb_rating,runtime,budget_adj,worldwide_gross_adj,Action,Adventure,Drama,Sci-Fi,Fantasy,Thriller,...,Horror,Music,War,Biography,Documentary,Short,tomatometer_score,audience_score_rt,original_language,MPA_Ratings
0,8.4,181,400000000,2797800564,1,1,1,1,0,0,...,0,0,0,0,0,0,94.0,90.0,English,PG-13
1,6.6,137,430707129,1188380976,1,1,0,0,1,0,...,0,0,0,0,0,0,33.0,54.0,English,PG-13
2,7.3,141,393679419,1504952267,1,1,0,1,0,0,...,0,0,0,0,0,0,76.0,83.0,English,PG-13
3,7.9,138,330043568,2226840424,1,1,0,1,0,0,...,0,0,0,0,0,0,93.0,85.0,English,PG-13
4,8.5,149,305280000,2084410886,1,1,0,1,0,0,...,0,0,0,0,0,0,85.0,91.0,English,PG-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3309,7.5,89,5682152,6223994,0,0,0,0,0,0,...,1,0,0,0,0,0,85.0,85.0,English,R
3310,6.1,90,6164349,1061902,0,0,0,0,0,1,...,0,0,1,0,0,0,45.0,45.0,English,R
3311,7.4,105,5399340,631182,0,0,1,0,0,0,...,0,1,0,0,0,0,64.0,83.0,English,R
3312,7.2,102,5566917,10142474,0,0,1,0,0,1,...,0,0,0,0,0,0,91.0,78.0,Spanish,NaN


In [566]:
model3_df.dropna(inplace=True)

In [570]:
model3_num = model3_df._get_numeric_data()
model3_num

,imdb_rating,runtime,budget_adj,worldwide_gross_adj,Action,Adventure,Drama,Sci-Fi,Fantasy,Thriller,...,History,Sport,Horror,Music,War,Biography,Documentary,Short,tomatometer_score,audience_score_rt
0,8.4,181,400000000,2797800564,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,94.0,90.0
1,6.6,137,430707129,1188380976,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,33.0,54.0
2,7.3,141,393679419,1504952267,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,76.0,83.0
3,7.9,138,330043568,2226840424,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,93.0,85.0
4,8.5,149,305280000,2084410886,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,85.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306,6.4,96,5486810,1179674,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,77.0,61.0
3307,5.7,118,5399340,1001118,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,67.0,36.0
3309,7.5,89,5682152,6223994,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,85.0,85.0
3310,6.1,90,6164349,1061902,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,45.0,45.0


In [571]:
model3_cat = model3_df.select_dtypes('object')
model3_cat

,original_language,MPA_Ratings
0,English,PG-13
1,English,PG-13
2,English,PG-13
3,English,PG-13
4,English,PG-13
...,...,...
3306,English,R
3307,English,R
3309,English,R
3310,English,R


In [572]:
model3_cat = pd.get_dummies(model3_cat, drop_first=True)

In [574]:
model3_combined = pd.concat([model3_num, model3_cat], axis=1)
model3_combined

,imdb_rating,runtime,budget_adj,worldwide_gross_adj,Action,Adventure,Drama,Sci-Fi,Fantasy,Thriller,...,original_language_Spanish (Spain),original_language_Swedish,original_language_Tamil,original_language_Thai,original_language_Unknown language,MPA_Ratings_NC-17,MPA_Ratings_PG,MPA_Ratings_PG-13,MPA_Ratings_R,MPA_Ratings_TVMA
0,8.4,181,400000000,2797800564,1,1,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,6.6,137,430707129,1188380976,1,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,7.3,141,393679419,1504952267,1,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,7.9,138,330043568,2226840424,1,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,8.5,149,305280000,2084410886,1,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306,6.4,96,5486810,1179674,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3307,5.7,118,5399340,1001118,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3309,7.5,89,5682152,6223994,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3310,6.1,90,6164349,1061902,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [587]:
linear_regression(model3_combined.drop('worldwide_gross_adj', axis=1), model3_combined['worldwide_gross_adj'], LogisticRegression())

C:\Users\Bosco\anaconda3\envs\DA_ENV\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train score is: 0.9008654602675059
Test score is: 0.0
Adjusted R2 is: -0.10051993067590992


ValueError: Data must be 1-dimensional